# Worksheet 08

Name:  Sungjun Park  
UID: U24688282

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [2]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[4.644731379209382, 2.604177479472395, 4.704088128195891, 4.782775911671455, 4.94710791914487, 5.387393660389889, 4.7169103306029765, 7.1991922038436975, 3.9275862055429176, 4.225675103785978]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [3]:

c2 = np.random.normal(8, 1, 10).tolist()
print(c2)

[7.963682829014255, 8.72514257158273, 7.73593920580918, 6.58091471302288, 6.795273939576605, 8.414848927644528, 8.255927747743302, 5.681175454963059, 9.770144563021795, 7.03436669521354]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [4]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[7.03436669521354, 9.770144563021795, 4.225675103785978, 3.9275862055429176, 7.1991922038436975, 4.7169103306029765, 5.387393660389889, 5.681175454963059, 4.94710791914487, 8.255927747743302]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

Mean and variance  
c1: 5, 1  
c2: 8, 1

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [5]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c2)

prob_c = [ len(c1) / (len(c1) + len(c2)) , len(c2) / (len(c1) + len(c2)) ]
mean = [ sum(c1)/len(c1) , sum(c2)/len(c2) ]

var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[0])**2, c2)) / len(c2) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[4.225675103785978, 3.9275862055429176, 4.7169103306029765, 5.387393660389889, 5.681175454963059, 4.94710791914487]
[7.03436669521354, 9.770144563021795, 7.1991922038436975, 8.255927747743302]
P(C_1) = 0.6,  P(C_2) = 0.4
mean_1 = 4.814308112404948,  mean_2 = 8.064907802455584
var_1 = 0.37329545173632034,  var_2 = 11.755348220077202


e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [6]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( pdf_i[0] * prob_c[0] / prob_x )
    prob_c1_x.append( pdf_i[1] * prob_c[1] / prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  7.03436669521354
probability of observing that point if it came from cluster 0 =  2.2313329216311835e-08
probability of observing that point if it came from cluster 1 =  0.03380692967805919
point =  9.770144563021795
probability of observing that point if it came from cluster 0 =  5.7109487564191744e-39
probability of observing that point if it came from cluster 1 =  0.03358189780160805
point =  4.225675103785978
probability of observing that point if it came from cluster 0 =  0.3082666311974018
probability of observing that point if it came from cluster 1 =  0.03217457164206774
point =  3.9275862055429176
probability of observing that point if it came from cluster 0 =  0.06362238608691899
probability of observing that point if it came from cluster 1 =  0.03189895458805173
point =  7.1991922038436975
probability of observing that point if it came from cluster 0 =  1.464879176594961e-09
probability of observing that point if it came from cluster 1 =  0.033845183240400784
point 

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [7]:
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x)/ len(prob_c1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x),   sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x) ]
# Calculate the updated variances
var = [sum(prob_c0_x[i] * (data[i] - mean[0]) ** 2 for i in range(len(data))) / sum(prob_c0_x),
       sum(prob_c1_x[i] * (data[i] - mean[1]) ** 2 for i in range(len(data))) / sum(prob_c1_x)]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.5344545144969807,  P(C_2) = 0.46554548550301933
mean_1 = 4.8180054829575125,  mean_2 = 7.6030018922058655
var_1 = 0.3409597566507591,  var_2 = 2.4131295952336926


g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [8]:
updated_prob_c0_x = []  # Updated P(C_0 | X_i)
updated_prob_c1_x = []  # Updated P(C_1 | X_i)

for p in data:
    pdf_i = []
    
    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        
    # P(X_i) = P(C_0) * P(X_i | C_0) + P(C_1) * P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # Updated P(C_j | X_i) = P(X_i | C_j) * P(C_j) / P(X_i)
    updated_prob_c0_x.append(pdf_i[0] * prob_c[0] / prob_x)
    updated_prob_c1_x.append(pdf_i[1] * prob_c[1] / prob_x)

# Printing updated probabilities
probs = zip(data, updated_prob_c0_x, updated_prob_c1_x)
for p in probs:
    print("Point = ", p[0])
    print("Updated Probability of coming from C_1 = " + str(p[1]))
    print("Updated Probability of coming from C_2 = " + str(p[2]))
    print()


Point =  7.03436669521354
Updated Probability of coming from C_1 = 5.576942052992846e-09
Updated Probability of coming from C_2 = 0.999999994423058

Point =  9.770144563021795
Updated Probability of coming from C_1 = 1.8956584079051233e-45
Updated Probability of coming from C_2 = 1.0

Point =  4.225675103785978
Updated Probability of coming from C_1 = 0.8271170676048204
Updated Probability of coming from C_2 = 0.17288293239517963

Point =  3.9275862055429176
Updated Probability of coming from C_1 = 0.46129432624025746
Updated Probability of coming from C_2 = 0.5387056737597427

Point =  7.1991922038436975
Updated Probability of coming from C_1 = 2.1132718305696898e-10
Updated Probability of coming from C_2 = 0.9999999997886727

Point =  4.7169103306029765
Updated Probability of coming from C_1 = 0.9408216508426056
Updated Probability of coming from C_2 = 0.059178349157394436

Point =  5.387393660389889
Updated Probability of coming from C_1 = 0.7543726426679248
Updated Probability of c

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [9]:
hard_assignments = [0 if prob_c0_x[i] > prob_c1_x[i] else 1 for i in range(len(data))]

# Printing the hard assignments
for i, label in enumerate(hard_assignments):
    print(f"Data point {data[i]} is assigned to Cluster {label}")


Data point 7.03436669521354 is assigned to Cluster 1
Data point 9.770144563021795 is assigned to Cluster 1
Data point 4.225675103785978 is assigned to Cluster 0
Data point 3.9275862055429176 is assigned to Cluster 0
Data point 7.1991922038436975 is assigned to Cluster 1
Data point 4.7169103306029765 is assigned to Cluster 0
Data point 5.387393660389889 is assigned to Cluster 0
Data point 5.681175454963059 is assigned to Cluster 0
Data point 4.94710791914487 is assigned to Cluster 0
Data point 8.255927747743302 is assigned to Cluster 1


### Clustering Aggregation

| Point | C | P |
|-------|---|---|
| A     | 0 | a |
| B     | 0 | b |
| C     | 2 | b |
| D     | 1 | c |
| E     | 1 | d |

a) Fill in the following table where for each pair of points determine whether C and P agree or disagree on how to cluster that pair.

| Pair | Disagreement |
|------|--------------|
| A  B |      Agreement      |
| A  C |      Disagreement       |
| A  D |      Disagreement     |
| A  E |      Disagreement     |
| B  C |      Agreement      |
| B  D |      Disagreement     |
| B  E |      Disagreement     |
| C  D |      Disagreement     |
| C  E |      Disagreement     |
| D  E |      Agreement      |


As datasets become very large, this process can become computationally challenging.

b) Given N points, what is the formula for the number of unique pairs of points one can create?

Number of Pairs=(N chooses 2)
 



Assume that clustering C clusters all points in the same cluster and clustering P clusters points as such:

| Point | P |
|-------|---|
| A     | 0 |
| B     | 0 |
| C     | 0 |
| D     | 1 |
| E     | 1 |
| F     | 2 |
| G     | 2 |
| H     | 2 |
| I     | 2 |

c) What is the maximum number of disagreements there could be for a dataset of this size? (use the formula from b)?

To calculate the maximum number of disagreements, we can use the formula for the number of unique pairs of points (as discussed in part b). Given that there are 9 points in total (N = 9), we can calculate the number of disagreements:



d) If we look at cluster 0. There are (3 x 2) / 2 = 3 pairs that agree with C (since all points in C are in the same cluster). For each cluster, determine how many agreements there are. How many total agreements are there? How many disagreements does that mean there are between C and P?

Cluster 0 in P (A, B, C):

Number of agreements: 3
Number of disagreements: 0
Cluster 1 in P (D, E):

Number of agreements: 1
Number of disagreements: 1
Cluster 2 in P (F, G, H, I):

Number of agreements: 6
Number of disagreements: -2
Total agreements: 10
Total disagreements: 1

e) Assuming that filtering the dataset by cluster number is a computationally easy operation, describe an algorithm inspired by the above process that can efficiently compute disagreement distances on large datasets.

For efficient computation of disagreement distances on large datasets, cluster the data, assign unique cluster identifiers, filter data for each cluster, and calculate agreements and disagreements within clusters. Sum these values across all clusters to determine total agreements and disagreements for the entire dataset. This approach optimizes efficiency by leveraging the clustered data's structure, reducing the need for exhaustive pairwise comparisons, making it suitable for large datasets.
